In [ ]:
import pandas as pd
import glob
import json
import dotted # https://pypi.org/project/dotted-notation/
import re
import matplotlib.pyplot as plt

from pathlib import Path
import seaborn as sns
import lib.datasciencetoolbelt as dstools
from lib.resultstorage import ResultStorage

In [ ]:
dstools.setup({
    "seaborn_context": "talk",
    "savefig": {
        "enable": True,
        "dir": Path("./postprocess_results"),
    }
})
result_storage = ResultStorage(Path("./results"))

#%matplotlib qt
%matplotlib inline


In [ ]:
result_storage_prefix = "itxg_bypass_v5"

id_vars__dottedpath_and_shortname_and_type = [
    ("zfs_setup.module_args.zfs.zfs_zil_itxg_bypass", "itxg_bypass", str),
    ("zfs_setup.module_args.zfs.zvol_request_sync", "zvol_request_sync", str),
    ("zfs_setup.module_args.zfs.zfs_zil_pmem_prb_ncommitters", "ncommitters", int),
    ("fio_config.fsync_every", "fsync_every", int),
    ("fio_config.numjobs", "numjobs", int)
]
id_vars = [p[1] for p in id_vars__dottedpath_and_shortname_and_type]

def extract_id_var_values(output_json):
    d = output_json
    id_var_values = {}
    for dp, sn, ty in id_vars__dottedpath_and_shortname_and_type: 
        v = dotted.get(d, dp)
        if not v:
            raise Exception(f"{d['file']}: dotted path {dp} not found")
        if sn in id_var_values:
            raise Exception(f"duplicate shortname {sn}")
        try:
            id_var_values[sn] = ty(v)
        except ValueError as e:
            raise Exception(f"cannot parse v={v!r}") from e
    return id_var_values

def get_fio_write_metrics(output_json):
    d = output_json
    jobs = dotted.get(d, "fio_jsonplus.jobs")
    assert len(jobs) == 1
    j0 = jobs[0]
    jw = jobs[0]["write"]
    return jw

def to_fio_results_dict(output_json):
    jw = get_fio_write_metrics(output_json)
    return {
        **extract_id_var_values(output_json),
        "w_iops_mean": jw["iops_mean"],
        "w_iops_stddev": jw["iops_stddev"],
        "w_lat_mean": dotted.get(jw, "lat_ns.mean"),
        "w_lat_stddev": dotted.get(jw, "lat_ns.stddev"),
    }

def to_kstat_results_dict(output_json):
    d = output_json
    return {
        **extract_id_var_values(output_json),
        **d["zvol_stats"],
        **d["itxg_bypass_stats"],
        **d["zil_pmem_stats"],
        **d["zil_pmem_ringbuf_stats"],
        "bio_total": d["zvol_stats"]["submit_bio__zvol_write(with_taskq_if_enabled)"],
        "taskq_delay": dotted.get(d, 'zvol_stats.zvol_write__taskq_qdelay'),
        "assign_aquire": dotted.get(d, 'itxg_bypass_stats.assign__aquisition_total'),
        "assign_vtable": dotted.get(d, 'itxg_bypass_stats.assign__vtable'),
        "assign_total": dotted.get(d, 'itxg_bypass_stats.assign__total'),
        "commit_total": dotted.get(d, 'itxg_bypass_stats.commit__total'),
        "commit_aquire": dotted.get(d, 'itxg_bypass_stats.commit__aquire'),
        
    }

def to_cpu_dict(output_json):
    d = output_json
    return {
        **extract_id_var_values(output_json),
        **{f"cpu_{comp}": val for comp, val in dotted.get(d, "cpu_time.allcpu").items()},
    }

In [ ]:
# compute `df_kstats`
rows = [to_kstat_results_dict(j) for j in result_storage.iter_results(result_storage_prefix)]
df_kstats = pd.DataFrame.from_dict(rows).set_index(id_vars).sort_index()

In [ ]:
# compute `df_cpu`
rows = [to_cpu_dict(j) for j in result_storage.iter_results(result_storage_prefix)]
df = pd.DataFrame.from_dict(rows)
df = df.set_index(id_vars).sort_index()
df = df.rename_axis("metric", axis=1)
df = df.stack()
df_cpu = df
del df
df_cpu

In [ ]:
## derive `df_cpu.notidle`
tmp = df_cpu.unstack("metric")
tmp["cpu_not_idle"] = tmp.sum(axis=1) - tmp.cpu_idle
df_cpu = tmp.stack()

In [ ]:
# compute `df`
rows = [to_fio_results_dict(j) for j in result_storage.iter_results(result_storage_prefix)]
df = pd.DataFrame.from_dict(rows)
df = df.set_index(id_vars).sort_index()
df = df.rename_axis("metric", axis=1)
df = df.stack()
df

In [ ]:
# a quick peek on the actual data in `df`
df.unstack("metric")

In [ ]:
# define df_zfssetup
data = df.unstack(["itxg_bypass", "zvol_request_sync", "ncommitters"])
data.columns = data.columns.map(lambda x: f"zil-pmem bypass={ {'1':'yes', '0': 'no'}[x[0]]} zvol_taskq={ {'1':'no', '0':'yes'}[x[1]] } ncommitters={x[2]}")
data = data.rename_axis("zfs_setup", axis=1)
data = data.stack()
data
df_zfssetup = data
del data

In [ ]:
id_vars

# Latency Breakdown

In [ ]:
import itertools

def filter_by_index_value(df, level, value):
    """Reduce dimensionality of a dataframe by filtering by and subsequently dropping one of its index levels.
    
    df is assumed to be a multi-indexed pd.DataFrame.
    First, filter the data frame so that we only keep rows whose index tuple has value `value` at level `level`.
    Now the resulting data frame only has a single value at the level.
    Thus remove that level from the index.
    Voila: dimensionality reduced.
    """
    df = df[df.index.get_level_values(level) == value]
    assert set(df.index.get_level_values(level)) == {value}
    df.index = df.index.droplevel(level)
    return df

def _test_filter_by_index_value():
    data = [{"favnum": n, "favletter": l, "id": id} for id, (n, l) in enumerate(itertools.product([23,42],["a", "b"]))]
    d = pd.DataFrame(data).set_index(["favnum", "favletter"])
    display(d)
    display(filter_by_index_value(d, "favnum", 23))
    display(filter_by_index_value(d, "favletter", "b"))
    
_test_filter_by_index_value()

In [ ]:
def level_values_sorted_unique(df, level):
    """Returns the sorted unique values of a DataFrame's multi-index at level `level`"""
    return sorted(list(set(df.index.get_level_values(level))))

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
class FactorizedDataFrameItem(AttrDict):
    @property
    def title(self):
        return f"{self.fdf.row}={self.rv}|{self.fdf.col}={self.cv}"
        
        
class FactorizedDataFrame:
    def __init__(self, data, row, col):
        self.data = data
        self.col = col
        self.row = row

        self.col_values = level_values_sorted_unique(self.data, self.col)
        self.row_values = level_values_sorted_unique(self.data, self.row)
        
    def iter_factorized(self):
        for ci, c in enumerate(self.col_values):
            for ri, r in enumerate(self.row_values):
                d = self.data.copy()
                d = filter_by_index_value(d, self.col, c)
                d = filter_by_index_value(d, self.row, r)
                # display(d)
            
                context = FactorizedDataFrameItem({
                    "fdf": self,
                    "d": d,
                    "ri": ri,
                    "rv": r,
                    "ci": ci,
                    "cv": c,
                })
                yield context
                

def factorplot(data=None, row=None, col=None, plot=None, subplots_kw={}):
    """Factorizez MultiIndex'ed DataFrame `data`, then invokes `plot` for each FactorizedDataFrameItem"""
    
    fdf = FactorizedDataFrame(data, row, col)
    
    subplots_kw = {
        "gridspec_kw": {'hspace': 1},
        **subplots_kw,
        "squeeze": False, # axes should always be two-dimensional
    }

    fig, axes = plt.subplots(len(fdf.row_values), len(fdf.col_values), **subplots_kw)

    for f in fdf.iter_factorized():
        ax = axes[f.ri, f.ci]
        ax.set_title(f.title)
        legend = f.ri == len(fdf.row_values)-1 and f.ci == len(fdf.col_values)-1
        plot(f.d, ax, legend)
        if legend:
            plt.legend(loc='lower left', bbox_to_anchor=(1,0.5))

In [ ]:
def filter_idvars(data):
    data = filter_by_index_value(data, 'zvol_request_sync', "1")
    data = filter_by_index_value(data, 'itxg_bypass', "1")
    data = data.query('ncommitters in [1,2,4,6,8,12,16,24] and fsync_every in [1,16] and numjobs <= 16')
    return data
    
def make_data(normalize=None):

    # compute latency breakdown variables
    data = df_kstats.copy()
    data['t_taskq'] = data.zvol_write__taskq_qdelay
    data['t_bypass_commit'] = data.commit_total
    data['t_bypass_assign_aquisition'] = data.assign__aquisition_total
    data['t_bypass_assign_exit'] = data.assign__exit
    data['t_prb'] = data.write_entry_time - data.prb_write__pmem
    data['t_pmem'] = data.prb_write__pmem
    data['t_get_data'] = data.get_data_time
    data['t_zil'] = data.assign__vtable - data.write_entry_time
    data['t_zvol_write'] = data.bio_total - (data.zvol_write__taskq_qdelay + data.zvol_write__1zil_commit + data.zvol_write__zvol_log_write_finish + data.zvol_write__2zil_commit)
    
    components = ['t_taskq', 't_bypass_commit', 't_bypass_assign_aquisition',
                  't_bypass_assign_exit', 't_prb', 't_pmem', 't_get_data', 't_zil', 't_zvol_write']
    
    # t_other is what we didn't account for yet
    data['t_other'] = data.bio_total - data[components].sum(axis=1)
    components += ['t_other']

    data = normalize(data)

    # project to components
    data = data[components]


    data = data.rename_axis('component', axis=1)
    data = data.stack()
    data = data.rename("rel_time")
    data = pd.DataFrame(data)

    data = filter_idvars(data)
    
    return data
    
    


xlim=(0,17)
xticks=range(0,17, 2)

####
print("relative latency breakdown")

def plot_latency_breakdown(data, ax, legend):
#     display(data)
    data = data.unstack("component")
    data.plot.area(ax=ax, legend=legend, xlim=xlim, xticks=xticks, ylim=(-0.1, 1.1))

data_lb = make_data(normalize=lambda data: data.div(data.bio_total, axis=0))
factorplot(data=data_lb, col='ncommitters', row='fsync_every', plot=plot_latency_breakdown,
          subplots_kw={
            "figsize": (30, 10),
            "gridspec_kw": {
                "hspace": 0.2,
            },
        })

plt.show()

####
print("""
Absolute latencies divided by numjobs, EXCEPT t_pmem.

Rationale: This section of the evaluation is about evaluating the PMEM write overhead limiter.
Reminder: Writing to PMEM above its write bandwidth limit wastes CPU time because the write
          instructions stall waiting for PMEM.
This means that an ideal implementation would allow t_pmem to reach the value where we achieve
peak performance (IOPS in our case), then keep the value at that level by queuing additional
writers on the CPU (we do it using a semaphore in t_prb).
Thus, an ideal t_pmem should look like so:

   _____________
  /
 /
/

If we divided t_pmem by `numjobs` like all the other `t_` metrics, that straight line would
need to turn into a declining line, proportional to `numjobs`.
That's quite hard to spot, so we felt it's better to not scale t_pmem at all.
""")

def plot_latency_curves(data, ax, legend):
#     display(data)
    data = data.unstack("component")
#     display(data)
    #data.plot.area(ax=ax, legend=legend, xlim=xlim, xticks=xticks)
    data.plot.line(ax=ax, legend=legend, xlim=xlim, xticks=xticks, ylim=(0, 3e10))
    
def normalize_by_numjobs_except_t_pmem(data):
    tmp = data.reset_index(level='numjobs')
    numjobs_orig = tmp.numjobs
    tmp = tmp.div(tmp.numjobs, axis=0)
    tmp['t_pmem'] = tmp['t_pmem'].mul(numjobs_orig) # !!!!!!!!!!!! don't normalize t_pmem
    tmp['numjobs'] = numjobs_orig
    tmp = tmp.set_index('numjobs', append=True)
    return tmp

data_abs = make_data(normalize=normalize_by_numjobs_except_t_pmem)
factorplot(data=data_abs, col='ncommitters', row='fsync_every', plot=plot_latency_curves,
          subplots_kw={
            "figsize": (30, 10),
            "gridspec_kw": {
                "hspace": 0.2,
            },
        })

plt.show()

####

def plot_perf_metric(metric):

    full_data = df.unstack("metric")
    full_data = filter_idvars(full_data)
    full_data = full_data[[metric]]
    ymax = full_data[metric].max() * 1.1

    def plot(data, ax, legend):
    #     display(data)
        #     data = data.pivot("numjobs", "metric", "value")
        #data.plot.line(ax=ax, legend=legend, xlim=(0, None), ylim=(0, context["full_data"][metric].max()))
        data.plot.line(ax=ax, legend=legend, xlim=xlim, xticks=xticks, ylim=(0, ymax))

    factorplot(data=full_data, col='ncommitters', row='fsync_every',
              plot=plot,
              subplots_kw={
                "figsize": (30, 5),
                "gridspec_kw": {
                    "hspace": 1,
                }
              })
    plt.show()

print("IOPS mean")
plot_perf_metric("w_iops_mean")
print("IOPS std")
plot_perf_metric("w_iops_stddev")

### Interpretation

- We only look at the lower row (`fsync_every = 16`) because that's where actual parallelism is available.
- We also only look at `numjobs \in [0, ..., 8]` as we only have 8 cores (16 SMTs).
- The PMEM overload problem is clearly visible:
  - We already achieve close-to-peak IOP performance at `ncommitters=2 and numjobs=6`.
  - And if we allow `ncommiters > 2` we can sustain those IOPS but at the cost of higher `t_pmem`.
  - That `t_pmem` time is time spent on-CPU, i.e., wasting CPU time.
    In comparison, for `ncommitters == 2` and rising `numjobs`, `t_prb` becomes larger, which is where we queue the threads waiting for a committer slot (i.e. the PMEM overload protection feature). (Look at the relative distribution and see how t_pmem shifts to t_prb as `numjobs` grows for `ncommitters == 2`.
- => the PMEM overload protection feature works.
- But the `itxg_bypass` implementation has poor multicore scalability.
  - `t_itxg_bypass*` are all divided by `numjobs`.
  - `t_itxg_bypass_assign_aquisition` rises as `numjobs` increase.
  - => `t_itxg_bypass_assign_aquisition` does not scale linearly with number of cores.
- The rising per-thread cost of the `itxg_bypass` slows `t_pmem` down at high `numjobs`.
  - Already visible for `numjobs \in [0, ..., 8]`
  - The eventual downturn of `t_pmem` at `numjobs > 8` might be caused by this as well (time-expenditure shifts to worse-and-worse scaling `itxg_bypass`, relieving pressure from PMEM)
  
=> conclusion for the next experiment:
- add a mode where the bypass's scaling behavior is avoided so that we can isolate the scalability of the PMEM overload protection feature
  - => add `itxg_bypass=2` where we do not use the rwlock at all.


# What overhead does the zvol taskq have?

In [ ]:
id_vars

In [ ]:
data = df_kstats.query('fsync_every in [1,4,16]')
data = data.query('zvol_request_sync == "0"')
data = data.reset_index()
# commit__total because it's essentially a single `mov` between aquisition and exit
data['rel_taskq_delay'] = data.zvol_write__taskq_qdelay / data.bio_total
g = sns.relplot(data=data,
            kind='line',
            height=5,
            ci='std',
            col='fsync_every',
            row='itxg_bypass',
#             hue='itxg_bypass',
            hue='ncommitters',
            markers=True,
            x ='numjobs', y='rel_taskq_delay')
g.set(ylim=(-0.1,1.1))

- This is relative overhead!
- In general: for both itxg_bypass values it's an unacceptably high overhead for something so simple
  => suggests that there is some scalability bottleneck
- The breakdowns for small ncommitters are due to the fact that waiting for a committer slot becomes the bottleneck in the whole equation.
- **A better viz would be repeating the latency breakdown graphs from the previous section but for the latencies involved with zvol_request_sync == 0** => needs different latency computation

# Some metrics in a different form (did'n't lead anywhere)

In [ ]:
 g = sns.relplot(
        data=filter_idvars(df.unstack('metric').reset_index()),
        kind='line',
        row='fsync_every',
        col='ncommitters',
        x='numjobs',
        y='w_iops_mean',
    )
g = sns.relplot(
        data=filter_idvars(df.unstack('metric').reset_index()),
        kind='line',
        row='fsync_every',
        col='ncommitters',
        x='numjobs',
        y='w_lat_mean',
    )

In [ ]:
 g = sns.relplot(
        data=filter_idvars(df.unstack('metric').reset_index()),
        kind='line',
        row='fsync_every',
        col='ncommitters',
        x='numjobs',
        y='w_iops_mean',
    )
g = sns.relplot(
        data=filter_idvars(df.unstack('metric').reset_index()),
        kind='line',
        row='fsync_every',
        col='ncommitters',
        x='numjobs',
        y='w_lat_mean',
    )

In [ ]:
id_vars

In [ ]:
df_kstats.columns

In [ ]:
def filter_data(data=None):
    data = data.copy()
    data = data.query("itxg_bypass == '1' and zvol_request_sync == '1' and fsync_every in [16]")
    data = data.query('ncommitters in [1,2,3,4,6,8,16] and numjobs <= 16')
    return data

def filter_and_plot(data=None, y=None):
    data = filter_data(data)
    data = data.reset_index()
    data = data.copy()
    data['fsync_every'] = data.fsync_every.astype('category')
    g = sns.relplot(
        data=data,
        kind='line',
#         col='fsync_every',
        col='ncommitters',
        x='numjobs',
        y=y,
        style='fsync_every', hue='fsync_every'
    )
    return g

sdf = df_kstats.copy()
sdf['get_committer_slot_lat'] =  sdf.prb_write__get_committer_slot / sdf.write_entry_count # FIXME: separate count metric
sdf['avg_pmem_write_lat'] = sdf.prb_write__pmem / sdf.write_entry_count # FIXME: separate count metric

pdf = df.copy()
pdf = pdf.unstack("metric")
pdf['rel_lat_pmem_vs_fio'] = sdf['avg_pmem_write_lat'] / pdf['w_lat_mean']
pdf['rel_lat_committerslot_vs_fio'] = sdf['get_committer_slot_lat'] / pdf['w_lat_mean']

tmp = pdf.reset_index(level='numjobs')
tmp['w_iops_mean_per_thread'] = tmp.w_iops_mean / tmp.numjobs
tmp = tmp.set_index('numjobs', append=True)
pdf['w_iops_mean_per_thread'] = tmp['w_iops_mean_per_thread']

pdf['iops_for_latency'] = pdf['w_iops_mean'] / pdf.w_lat_mean

cdf = df_cpu.copy().unstack('metric')
cdf['cpu_total'] = cdf.cpu_not_idle + cdf.cpu_idle
cdf['cpu_isolcpus_idle'] = cdf.cpu_total / 2 # 50% of cores were disabled using isolcpus= => their time doesn't count
cdf['cpu_nonisol_total'] = cdf.cpu_total - cdf.cpu_isolcpus_idle
cdf['cpu_nonisol_utilization'] = cdf.cpu_not_idle / cdf.cpu_nonisol_total

display(cdf.cpu_nonisol_utilization.describe())
print("note: SMT is still active, thus 50% or more _could_ mean 100% busy bars for the SMT thread of one core in htop. In practice this _is_ always the case.")

pdf['cpu_time_per_iops'] = 1e9 * ((cdf.cpu_not_idle) / pdf.w_iops_mean)

tmp = pdf.reset_index(level='numjobs')
tmp['cpu_time_per_iops_by_thread'] = tmp.cpu_time_per_iops / tmp.numjobs
tmp = tmp.set_index('numjobs', append=True)
pdf['cpu_time_per_iops_by_thread'] = tmp.cpu_time_per_iops_by_thread

tmp = cdf.reset_index(level='numjobs')
tmp['cpu_time_per_thread'] = tmp.cpu_not_idle / tmp.numjobs
tmp = tmp.set_index('numjobs', append=True)
cdf['cpu_time_per_thread'] = tmp.cpu_time_per_thread

# sdf['pmem_time_per_iops'] = ((sdf.prb_write__pmem) / (pdf.w_iops_mean))
sdf['pmem_time_per_cpu_nonisol_total'] = (sdf.prb_write__pmem / 1e9) / (cdf.cpu_nonisol_total)
# sdf['bypass_time_per_cpu_nonisol_total'] = ((df_latencybreakdown.t_bypass_assign_aquisition + df_latencybreakdown.t_bypass_assign_exit + df_latencybreakdown.t_bypass_commit)/1e9) / cdf.cpu_nonisol_total

g = filter_and_plot(data=cdf, y='cpu_nonisol_utilization')
g.set(ylim=(-0.1, 1.1))
g = filter_and_plot(data=sdf, y='pmem_time_per_cpu_nonisol_total')
g.set(ylim=(-0.1, 1.1))


filter_and_plot(data=pdf, y='w_iops_mean')
#filter_and_plot(data=pdf, y='w_iops_mean_per_thread')

g = filter_and_plot(data=pdf, y='cpu_time_per_iops_by_thread')
g.set(ylim=(0, None))
g = filter_and_plot(data=cdf, y='cpu_time_per_thread')
g.set(ylim=(0, None))

# filter_and_plot(data=sdf, y='pmem_time_per_iops')
# g = filter_and_plot(data=sdf, y='bypass_time_per_cpu_nonisol_total')
# g.set(ylim=(-0.1, 1.1))


filter_and_plot(data=sdf, y='avg_pmem_write_lat')


# filter_and_plot(data=pdf, y='iops_for_latency')

# g = filter_and_plot(data=pdf, y='rel_lat_pmem_vs_fio')
# g.set(ylim=(-0.1,0.5))

# g = filter_and_plot(data=pdf, y='rel_lat_committerslot_vs_fio')
# g.set(ylim=(-0.1,0.5))

- `cpu_time_per_iops` and `avg_pmem_write_lat` have correlating edges. These edges move toward the right as `ncommitters` increases.
  - Both curves smoothen as `ncommitters` grows between the lower left and the edge point.
  - With this hardware configuration (single DIMM) A value of `ncommitters` between 1 and 2 yields near optimal performance (>300k IOPS) while not wasting CPU time on PMEM stall
  - For higher `ncommitters` 